# Tare Dash 
## Visualización avanzada 

### Manel Soler Sanz 

### Resumen de propiedades del Dash

* Capacidad de importar uno o varios archivos (xls o csv)
* Tabla para visualizar los datos
* 4 tipos de graficos : Barras, puntos, Area, Funnel(embudo)
* Elección de ejes entre todas las columnas . Numéricas o no numéricas
* Opción de escala logaritmica

In [2]:
import base64
import datetime
import io
import numpy as np
from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
from dash import dcc
from dash import html
from dash import dash_table
import plotly.express as px

import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets,
                suppress_callback_exceptions=True)

app.layout = html.Div([ # this code section taken from Dash docs https://dash.plotly.com/dash-core-components/upload
    dcc.Upload(
        id='upload-data',
        children=[html.H1(children='Trabajo Visualización por Manel Soler'),
            html.Div([
            'Arrastra o selecciona uno o varios (csv o xls) ',
            html.A('Archivos')
        ])],
        style={
            'width': '1200px',
            'height': '500px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderColor': 'red',
            'borderTopColor':'white',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '80px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-div'),
    html.Div(id='output-datatable'),
])


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),
        
        # preguntamos por el tipo de gráfico deseado. Tenemos 4 ocpiones : Barras, puntos, Area, Funnel(embudo)
        html.P("Elige tipo gráfico"),
        dcc.RadioItems(id="graph-selected",
                       options = [{"label": "Gráfico Barras", "value" : "bar"},
                                {'label': "Gráfico de puntos", "value" : "scatter"},
                                 {'label': "Gráfico de Area", "value" : "area"},
                                 {'label': "Gráfico de Funnel(Embudo)", "value" : "funnel"}],
                       value = "bar"
        ),
        
        # preguntams por los ejes x e y para la representación y también la escala deseada
        html.P("Inserta X axis data"),
        dcc.Dropdown(id='xaxis-data',
                     options=[{'label':x, 'value':x} for x in df.columns]),
        html.P("Elige escala"),
        dcc.RadioItems(id="xscale-selected",
                       options = [{"label": "Lineal", "value" : "linear"},
                                {'label': "Log", "value" : "log"}],
                       value = "linear"
        ),
        
        html.P("Inserta Y axis data"),
        dcc.Dropdown(id='yaxis-data',
                     options=[{'label':x, 'value':x} for x in df.columns]),
        html.P("Elige escala"),
        dcc.RadioItems(id="yscale-selected",
                       options = [{"label": "Lineal", "value" : "linear"},
                                {'label': "Log", "value" : "log"}],
                       value = "linear"
        ),
        
        html.Button(id="submit-button", children="Crear Grafico"),
        html.Hr(),
        # añado tabla para visaulizar los datos y poder ver que valores tiene cada columna
        html.H2(children='Tabla para visualizar los datos : '),
        dash_table.DataTable(
            style_cell={
            'width': '400px',
            'height': '400xp'},
            style_table={'width': '1900px', 'overflowX': 'auto'},
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns],
            page_size=10
        ),
        dcc.Store(id='stored-data', data=df.to_dict('records')),
        #html.H2(children='Tabla: '),
        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])


@app.callback(Output('output-datatable', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


@app.callback(Output('output-div', 'children'),
              Input('submit-button','n_clicks'),
              State('graph-selected', 'value'),
              State('stored-data','data'),
              State('xaxis-data','value'),
              State('xscale-selected', 'value'),              
              State('yaxis-data', 'value'),
              State('yscale-selected', 'value'))
def make_graphs(n,graph_chosen, data, x_data,x_scale, y_data,y_scale):
    if n is None:
        return no_update
    logx = False # si queremos escalas logaritmicas , esto tomará valor True
    logy = False
    if x_scale == "log": 
        logx = True
    if y_scale == "log":
        logy = True
    if graph_chosen == "bar":      
        bar_fig = px.bar(data, x=x_data, y=y_data,log_x=logx, log_y = logy)
        return dcc.Graph(figure=bar_fig)
    elif graph_chosen == "scatter" :
        scatter_fig = px.scatter(data, x=x_data, y=y_data, log_x=logx, log_y = logy)
        return dcc.Graph(figure=scatter_fig)
    elif graph_chosen == "area" :
        area_fig = px.area(data, x=x_data, y=y_data, log_x=logx, log_y = logy)
        return dcc.Graph(figure=area_fig)
    elif graph_chosen == "funnel" :
        funnel_fig = px.funnel(data, x=x_data, y=y_data, log_x=logx, log_y = logy)
        return dcc.Graph(figure=funnel_fig)
    

     


app.run_server(mode='external', port=2543)

Dash app running on http://127.0.0.1:2443/


C:\Users\34695\anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()
